In [1]:
import os

# Set various TF options
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

import tensorflow as tf
import keras.api as keras

print("TF version:", tf.__version__)
print("TF device:", tf.config.list_physical_devices('GPU')) # or 'CPU'

TF version: 2.16.1
TF device: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [ ]:
from common import CV_DATA_DIR
from pathlib import Path

IMAGE_SHAPE = (128, 128, 3)
DATASET_PATH = Path(CV_DATA_DIR, 'homer_bart_2.zip')

archive = keras.utils.get_file(origin=DATASET_PATH.as_uri(), extract=True)
data_dir = Path(archive).with_suffix('')

train_ds = keras.utils.image_dataset_from_directory(
    Path(data_dir, 'training_set'),
    image_size=IMAGE_SHAPE[:2],
    batch_size=8,
    labels='inferred',
    shuffle=True
)
val_ds = keras.utils.image_dataset_from_directory(
    Path(data_dir, 'test_set'),
    image_size=IMAGE_SHAPE[:2],
    batch_size=1,
    labels='inferred',
    shuffle=False
)

In [ ]:
inputs = keras.layers.Rescaling(scale=1./255)(keras.Input(shape=IMAGE_SHAPE))

# Setting `weights` to 'imagenet' will use particular dataset
# Setting `include_top` to False implies creating own dense layer for custom objects classification 
base_model = keras.applications.ResNet50(weights='imagenet',
                                         include_top=False, 
                                         input_tensor=inputs)

# Freeze training of base model
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
hidden_layer_neurons = int((2048 + 2) / 2)

head_model = base_model.output
head_model = keras.layers.GlobalAveragePooling2D()(head_model)
head_model = keras.layers.Dense(hidden_layer_neurons, activation='relu')(head_model)
head_model = keras.layers.Dropout(rate = 0.2)(head_model)
head_model = keras.layers.Dense(hidden_layer_neurons, activation='relu')(head_model)
head_model = keras.layers.Dropout(rate = 0.2)(head_model)
head_model = keras.layers.Dense(2, activation='softmax')(head_model)

In [ ]:
network = keras.models.Model(inputs = base_model.input, outputs = head_model)
network.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = network.fit(train_ds, epochs=50, validation_data=val_ds)

In [ ]:
import numpy as np
predictions = np.argmax(network.predict(val_ds), axis = 1)

In [57]:
classes = []
for image, clazz in val_ds.as_numpy_iterator():
    classes.append(clazz)
classes = np.asarray(classes)

2024-06-03 21:42:25.907239: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [58]:
from sklearn.metrics import accuracy_score
accuracy_score(classes, predictions)

0.7407407407407407